# Chapter 17: Advanced Topics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_17_advanced_topics.ipynb)

**Advanced rendering topics** cover cutting-edge techniques used in production renderers. This chapter explores photon mapping, bidirectional path tracing, Metropolis light transport, and modern denoising methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List, Optional
from dataclasses import dataclass

## 1. Photon Mapping

**Photon mapping** is a two-pass algorithm for global illumination.

### Pass 1: Photon Tracing

Emit photons from lights and trace through scene:

1. Emit photon from light with power $\Phi$
2. Trace through scene, storing at diffuse surfaces
3. Russian roulette for absorption/reflection
4. Build photon map (k-d tree)

### Pass 2: Rendering

Ray trace from camera, estimating radiance using photon map:

$$
L_r(\mathbf{x}, \omega_o) \approx \frac{1}{\pi r^2} \sum_{p=1}^n \Phi_p \cdot f_r(\mathbf{x}, \omega_p, \omega_o)
$$

where:
- $n$ = number of nearest photons
- $r$ = radius containing $n$ photons
- $\Phi_p$ = photon power
- $f_r$ = BRDF

### Caustic Photon Map

Separate map for specular→diffuse paths (caustics):
- More photons concentrated in caustic regions
- Smaller search radius for sharper caustics

In [ ]:
@dataclass
class Photon:
    """Photon with position, direction, and power"""
    position: np.ndarray  # 3D position
    direction: np.ndarray  # Incoming direction
    power: np.ndarray  # RGB power

class PhotonMap:
    """Simple photon map with k-nearest neighbor search"""
    def __init__(self, max_photons: int = 100000):
        self.photons = []
        self.max_photons = max_photons
    
    def store(self, photon: Photon):
        """Store photon in map"""
        if len(self.photons) < self.max_photons:
            self.photons.append(photon)
    
    def find_nearest(self, position: np.ndarray, max_dist: float, max_photons: int) -> List[Photon]:
        """Find nearest photons (simple linear search for demo)"""
        distances = []
        for photon in self.photons:
            dist = np.linalg.norm(photon.position - position)
            if dist < max_dist:
                distances.append((dist, photon))
        
        distances.sort(key=lambda x: x[0])
        return [p for _, p in distances[:max_photons]]
    
    def estimate_radiance(self, position: np.ndarray, normal: np.ndarray, 
                         max_dist: float = 0.5, num_photons: int = 50) -> np.ndarray:
        """Estimate radiance at surface point"""
        photons = self.find_nearest(position, max_dist, num_photons)
        
        if len(photons) == 0:
            return np.zeros(3)
        
        # Find radius of search sphere
        max_r = 0
        for p in photons:
            r = np.linalg.norm(p.position - position)
            max_r = max(max_r, r)
        
        # Density estimation
        radiance = np.zeros(3)
        for p in photons:
            # Lambertian BRDF: cos(theta) / pi
            cos_theta = max(0, np.dot(normal, -p.direction))
            radiance += p.power * cos_theta
        
        # Normalize by area
        area = math.pi * max_r * max_r
        if area > 0:
            radiance /= area
        
        return radiance

print("✓ Photon mapping loaded")

## 2. Bidirectional Path Tracing (BDPT)

**BDPT** traces paths from both camera and light, connecting them.

### Algorithm

1. **Eye subpath**: Trace from camera $s_0, s_1, ..., s_k$
2. **Light subpath**: Trace from light $t_0, t_1, ..., t_l$
3. **Connect**: Try all combinations $(s_i, t_j)$
4. **MIS weights**: Combine using multiple importance sampling

### Path Contribution

For connected path $\bar{\mathbf{x}}_{s,t}$ with $s$ eye vertices and $t$ light vertices:

$$
C_{s,t} = \alpha_{s,t} \cdot f(\bar{\mathbf{x}}_{s,t}) \cdot G(s_s \leftrightarrow t_t) \cdot w_{s,t}
$$

where:
- $\alpha_{s,t}$ = throughput product
- $f$ = path contribution (light × BRDF × geometry)
- $G$ = connection geometry term
- $w_{s,t}$ = MIS weight

### MIS Weight (Balance Heuristic)

$$
w_{s,t} = \frac{p_{s,t}}{\sum_{i=0}^{s+t} p_i}
$$

Optimally combines all sampling strategies.

In [ ]:
# BDPT conceptual implementation
class PathVertex:
    """Vertex in a light transport path"""
    def __init__(self, position, normal, throughput, pdf):
        self.position = position  # 3D position
        self.normal = normal      # Surface normal
        self.throughput = throughput  # Path throughput
        self.pdf = pdf  # Path PDF

def trace_eye_subpath(origin, direction, max_depth=5):
    """Trace subpath from camera (conceptual)"""
    path = []
    # Start from camera
    # Trace through scene, building vertex list
    # Each bounce adds a vertex with position, normal, throughput, PDF
    return path

def trace_light_subpath(light_position, max_depth=5):
    """Trace subpath from light (conceptual)"""
    path = []
    # Start from light
    # Emit in random direction
    # Trace through scene, building vertex list
    return path

def connect_bdpt(eye_path, light_path, s, t):
    """Connect s-th eye vertex with t-th light vertex (conceptual)"""
    if s >= len(eye_path) or t >= len(light_path):
        return 0
    
    eye_vertex = eye_path[s]
    light_vertex = light_path[t]
    
    # Check visibility
    # Compute geometry term
    # Evaluate BRDFs
    # Compute MIS weight
    # Return path contribution
    
    return 0  # Placeholder

def bidirectional_path_trace(camera_ray, scene):
    """Full BDPT pixel estimate (conceptual)"""
    radiance = np.zeros(3)
    
    # Trace eye subpath
    eye_path = trace_eye_subpath(camera_ray.origin, camera_ray.direction)
    
    # Trace light subpath
    light_path = trace_light_subpath(scene.lights[0].position)
    
    # Try all connections (s,t) where s+t <= max_depth
    max_depth = 5
    for s in range(len(eye_path) + 1):
        for t in range(len(light_path) + 1):
            if s + t > max_depth:
                continue
            contribution = connect_bdpt(eye_path, light_path, s, t)
            radiance += contribution
    
    return radiance

print("✓ BDPT concepts loaded")

## 3. Metropolis Light Transport (MLT)

**MLT** uses Markov Chain Monte Carlo to explore path space.

### Metropolis-Hastings Algorithm

1. Start with seed path $\bar{\mathbf{x}}_0$
2. Propose mutation $\bar{\mathbf{x}}' = T(\bar{\mathbf{x}}_i)$
3. Accept with probability:
$$
a(\bar{\mathbf{x}}_i \to \bar{\mathbf{x}}') = \min\left(1, \frac{f(\bar{\mathbf{x}}') T(\bar{\mathbf{x}}' \to \bar{\mathbf{x}}_i)}{f(\bar{\mathbf{x}}_i) T(\bar{\mathbf{x}}_i \to \bar{\mathbf{x}}')}\right)
$$
4. If accepted: $\bar{\mathbf{x}}_{i+1} = \bar{\mathbf{x}}'$, else: $\bar{\mathbf{x}}_{i+1} = \bar{\mathbf{x}}_i$

### Mutation Strategies

- **Small step**: Perturb existing vertex positions
- **Large step**: Generate completely new path
- **Caustic perturbation**: Preserve specular sub-paths
- **Lens perturbation**: Change camera ray direction

### Advantages

- Automatic importance sampling
- Excellent for difficult light transport (caustics, indirect)
- Explores path space efficiently

### Disadvantages

- Slow startup
- Can produce structured noise
- Difficult to parallelize

In [ ]:
# MLT conceptual framework
class Path:
    """Light transport path"""
    def __init__(self, vertices, contribution):
        self.vertices = vertices  # List of vertices
        self.contribution = contribution  # f(path)

def mutate_small_step(path):
    """Small perturbation mutation"""
    # Perturb random vertex position slightly
    # Recompute affected sub-path
    # Return new path
    return path  # Placeholder

def mutate_large_step(scene):
    """Large step mutation - generate new path"""
    # Generate completely new path
    # Trace from camera with random direction
    return None  # Placeholder

def metropolis_sample(scene, num_mutations=10000):
    """Metropolis light transport (conceptual)"""
    # Generate initial path
    current_path = None  # Start with seed
    samples = []
    
    for i in range(num_mutations):
        # Choose mutation type
        if random.random() < 0.3:  # 30% large step
            proposed = mutate_large_step(scene)
            T_forward = 0.3
            T_backward = 0.3
        else:  # 70% small step
            proposed = mutate_small_step(current_path)
            T_forward = 0.7
            T_backward = 0.7
        
        # Metropolis acceptance
        if proposed is None:
            continue
        
        # Acceptance probability
        a = min(1.0, (proposed.contribution * T_backward) / 
                     (current_path.contribution * T_forward))
        
        if random.random() < a:
            current_path = proposed
        
        samples.append(current_path)
    
    return samples

print("✓ MLT concepts loaded")

## 4. Image-Based Lighting (IBL)

**IBL** uses environment maps for lighting.

### Environment Map Sampling

1. **Uniform sampling**: Sample random direction on sphere
2. **Importance sampling**: Sample proportional to luminance
3. **Stratified sampling**: Divide sphere into strata

### Importance Sampling Strategy

Build 2D CDF from environment map:

$$
p(\theta, \phi) = \frac{L(\theta, \phi) \sin\theta}{\int_0^{2\pi} \int_0^\pi L(\theta', \phi') \sin\theta' \, d\theta' d\phi'}
$$

Sample using inversion method.

### Spherical Harmonics (SH) Lighting

Represent environment as SH coefficients:

$$
L(\mathbf{d}) \approx \sum_{l=0}^n \sum_{m=-l}^l c_{lm} Y_{lm}(\mathbf{d})
$$

Fast evaluation, good for diffuse lighting.

In [ ]:
def sample_environment_map_uniform(env_map):
    """Uniformly sample direction on sphere"""
    # Random spherical coordinates
    u = random.random()
    v = random.random()
    
    theta = math.acos(1 - 2 * u)  # [0, pi]
    phi = 2 * math.pi * v  # [0, 2pi]
    
    # Convert to Cartesian
    x = math.sin(theta) * math.cos(phi)
    y = math.sin(theta) * math.sin(phi)
    z = math.cos(theta)
    
    direction = np.array([x, y, z])
    
    # Look up environment map
    u_tex = phi / (2 * math.pi)
    v_tex = theta / math.pi
    
    # Sample env map at (u_tex, v_tex)
    # ...
    
    return direction, 1.0 / (4 * math.pi)  # direction, pdf

def build_env_map_cdf(env_map):
    """Build cumulative distribution for importance sampling"""
    height, width = env_map.shape[:2]
    
    # Compute luminance for each pixel
    luminance = np.zeros((height, width))
    for j in range(height):
        theta = (j + 0.5) / height * math.pi
        sin_theta = math.sin(theta)
        for i in range(width):
            rgb = env_map[j, i]
            lum = 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]
            luminance[j, i] = lum * sin_theta
    
    # Build marginal CDF (rows)
    marginal = np.sum(luminance, axis=1)
    marginal_cdf = np.cumsum(marginal)
    marginal_cdf /= marginal_cdf[-1]
    
    # Build conditional CDF (columns given row)
    conditional_cdf = np.cumsum(luminance, axis=1)
    for j in range(height):
        if conditional_cdf[j, -1] > 0:
            conditional_cdf[j] /= conditional_cdf[j, -1]
    
    return marginal_cdf, conditional_cdf

print("✓ IBL sampling loaded")

## 5. Denoising Techniques

Modern renderers use **denoising** to reduce Monte Carlo noise.

### Edge-Avoiding À-Trous Wavelet

Multi-scale filter preserving edges:

$$
I^{(l+1)}(\mathbf{p}) = \sum_{\mathbf{q} \in \Omega} h(\mathbf{q}) \cdot w(\mathbf{p}, \mathbf{q}) \cdot I^{(l)}(\mathbf{p} + 2^l \mathbf{q})
$$

Weight function:
$$
w(\mathbf{p}, \mathbf{q}) = w_c \cdot w_n \cdot w_d
$$

- $w_c$: color similarity
- $w_n$: normal similarity  
- $w_d$: depth similarity

### SVGF (Spatiotemporal Variance-Guided Filtering)

1. **Temporal accumulation**: Blend with previous frame
2. **Variance estimation**: Track sample variance
3. **Spatial filtering**: À-trous with variance-based kernel
4. **Edge-stopping**: Use G-buffer (normals, depth, albedo)

### Neural Denoising

Machine learning approaches:
- Train CNN on noisy/clean pairs
- Use auxiliary features (normals, albedo, depth)
- Real-time capable (DLSS, OptiX denoiser)

In [ ]:
def gaussian_kernel_1d(sigma, size):
    """1D Gaussian kernel"""
    kernel = np.zeros(size)
    center = size // 2
    sum_val = 0
    
    for i in range(size):
        x = i - center
        kernel[i] = math.exp(-(x * x) / (2 * sigma * sigma))
        sum_val += kernel[i]
    
    return kernel / sum_val

def bilateral_filter_simple(image, sigma_spatial=2.0, sigma_range=0.1):
    """Simple bilateral filter for denoising"""
    height, width = image.shape[:2]
    result = np.zeros_like(image)
    
    kernel_size = int(3 * sigma_spatial)
    
    print(f"  Processing {height}x{width} image with kernel size {kernel_size*2+1}x{kernel_size*2+1}")
    
    for y in range(height):
        if y % 64 == 0:
            progress = (y / height) * 100
            print(f"  Progress: {progress:.1f}% (row {y}/{height})")
        
        for x in range(width):
            center_color = image[y, x]
            
            sum_weight = 0
            sum_color = np.zeros(3)
            
            for dy in range(-kernel_size, kernel_size + 1):
                for dx in range(-kernel_size, kernel_size + 1):
                    ny = np.clip(y + dy, 0, height - 1)
                    nx = np.clip(x + dx, 0, width - 1)
                    
                    neighbor_color = image[ny, nx]
                    
                    # Spatial weight
                    spatial_dist2 = dx * dx + dy * dy
                    w_spatial = math.exp(-spatial_dist2 / (2 * sigma_spatial * sigma_spatial))
                    
                    # Range weight (color similarity)
                    color_dist2 = np.sum((center_color - neighbor_color) ** 2)
                    w_range = math.exp(-color_dist2 / (2 * sigma_range * sigma_range))
                    
                    weight = w_spatial * w_range
                    
                    sum_weight += weight
                    sum_color += weight * neighbor_color
            
            if sum_weight > 0:
                result[y, x] = sum_color / sum_weight
            else:
                result[y, x] = center_color
    
    print("  Progress: 100.0% (complete)")
    return result

print("✓ Denoising loaded")

## Example: Simple Denoising Demo

In [ ]:
# Create test image with noise
def create_noisy_gradient():
    """Create smooth gradient with Monte Carlo noise"""
    size = 256
    image = np.zeros((size, size, 3))
    
    print("Creating noisy test image...")
    for y in range(size):
        if y % 64 == 0:
            print(f"  Generating row {y}/{size}")
        for x in range(size):
            # Smooth gradient
            base_color = np.array([
                x / size,
                y / size,
                (x + y) / (2 * size)
            ])
            
            # Add Monte Carlo noise (simulating low sample count)
            noise = np.random.normal(0, 0.1, 3)
            
            image[y, x] = np.clip(base_color + noise, 0, 1)
    
    print("✓ Noisy image created")
    return image

# Create noisy image
print("="*70)
print("DENOISING DEMONSTRATION")
print("="*70)
np.random.seed(42)
noisy = create_noisy_gradient()

# Denoise
print("\nApplying bilateral filter denoising...")
print("  Parameters: sigma_spatial=3.0, sigma_range=0.15")
denoised = bilateral_filter_simple(noisy, sigma_spatial=3.0, sigma_range=0.15)
print("✓ Denoising complete!")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(noisy)
axes[0].set_title('Noisy Rendering (Low SPP)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(denoised)
axes[1].set_title('Bilateral Filtered (Denoised)', fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Bilateral filter successfully removes noise while preserving edges!")
print("="*70)

## Summary

**Advanced rendering techniques** used in production:

### Key Concepts

1. **Photon Mapping**
   - Two-pass algorithm
   - Excellent for caustics
   - k-d tree photon storage
   - Density estimation

2. **Bidirectional Path Tracing (BDPT)**
   - Traces from camera AND light
   - All path lengths simultaneously
   - Multiple importance sampling (MIS)
   - Better than unidirectional PT for complex scenes

3. **Metropolis Light Transport (MLT)**
   - Markov Chain Monte Carlo
   - Automatic importance sampling
   - Explores difficult paths
   - Mutation strategies

4. **Image-Based Lighting (IBL)**
   - Environment maps for lighting
   - Importance sampling
   - Spherical harmonics
   - Fast diffuse lighting

5. **Denoising**
   - Bilateral filter: edge-preserving
   - À-trous wavelet: multi-scale
   - SVGF: spatiotemporal
   - Neural denoising: ML-based

### Production Usage

- **Photon Mapping**: Caustics in glass, water
- **BDPT**: Complex indirect lighting
- **MLT**: Difficult scenes (e.g., light through keyhole)
- **IBL**: Outdoor scenes, studio lighting
- **Denoising**: All real-time ray tracing

### Modern Renderers

- **Arnold**: BDPT + importance sampling
- **V-Ray**: Multiple algorithms selectable
- **RenderMan**: Path tracing + importance sampling
- **Cycles**: Unidirectional PT + MIS
- **Real-time**: Ray tracing + denoising (DLSS, SVGF)

These advanced techniques enable production-quality rendering for film, games, and visualization.